In [1]:
import pandas as pd
import sqlite3

In [2]:
orders = pd.read_csv("orders.csv")


In [3]:
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users=pd.read_json("users.json")

In [5]:
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
# Create SQLite database in memory
conn = sqlite3.connect(":memory:")

# Read SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

# Execute SQL script
conn.executescript(sql_script)

# Load table into pandas DataFrame
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [7]:
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [8]:
orders_users = pd.merge(
    orders,
    users,
    how="left",
    left_on="user_id",
    right_on="user_id"
)

In [9]:
final_dataset = pd.merge(
    orders_users,
    restaurants,
    how="left",
    left_on="restaurant_id",
    right_on="restaurant_id"
)

In [10]:
final_dataset.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
final_dataset.shape

(10000, 12)

In [12]:
final_dataset.to_csv("final_food_delivery_dataset.csv", index=False)


In [15]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")


In [28]:
gold_df = df[df["membership"] == "Gold"]

gold_df.groupby("city")["total_amount"].sum()





city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

In [29]:
df.groupby("cuisine")["total_amount"].mean()


cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64

In [16]:
user_total = df.groupby("user_id")["total_amount"].sum()

count_users = user_total[user_total > 1000].count()
count_users

np.int64(2544)

In [24]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating'],
      dtype='object')

In [27]:
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[3.0, 3.5, 4.0, 4.5, 5.0],
    labels=["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"],
    include_lowest=True
)
df.groupby("rating_range")["total_amount"].sum()


C:\Users\srini\AppData\Local\Temp\ipykernel_21880\1630969756.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("rating_range")["total_amount"].sum()


rating_range
3.0–3.5    2136772.70
3.6–4.0    1717494.41
4.1–4.5    1960326.26
4.6–5.0    2197030.75
Name: total_amount, dtype: float64

In [30]:
gold_df = df[df["membership"] == "Gold"]

gold_df.groupby("city")["total_amount"].mean()


city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64

In [31]:
restaurant_count = df.groupby("cuisine")["restaurant_id"].nunique()
restaurant_count
revenue_per_cuisine = df.groupby("cuisine")["total_amount"].sum()
revenue_per_cuisine
summary = pd.concat([restaurant_count, revenue_per_cuisine], axis=1)
summary.columns = ["distinct_restaurants", "total_revenue"]

summary.sort_values("distinct_restaurants")


,distinct_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [32]:
total_orders = len(df)
gold_orders = len(df[df["membership"] == "Gold"])

percentage = (gold_orders / total_orders) * 100
round(percentage)


50

In [33]:
restaurant_stats = df.groupby("restaurant_name_y").agg(
    total_orders=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)
filtered = restaurant_stats[restaurant_stats["total_orders"] < 20]
filtered.sort_values("avg_order_value", ascending=False).head()


,total_orders,avg_order_value
restaurant_name_y,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625


In [34]:
df[df["restaurant_name_y"] == "Restaurant_294"][["restaurant_name_y"]].head()


,restaurant_name_y
1407,Restaurant_294
1643,Restaurant_294
2426,Restaurant_294
3174,Restaurant_294
3243,Restaurant_294


In [35]:
df[df["restaurant_name_y"] == "Restaurant_294"][["restaurant_name_x", "restaurant_name_y"]].head()


,restaurant_name_x,restaurant_name_y
1407,Hotel Dhaba Multicuisine,Restaurant_294
1643,Hotel Dhaba Multicuisine,Restaurant_294
2426,Hotel Dhaba Multicuisine,Restaurant_294
3174,Hotel Dhaba Multicuisine,Restaurant_294
3243,Hotel Dhaba Multicuisine,Restaurant_294


In [36]:
df[df["restaurant_name_y"] == "Restaurant_294"][
    ["restaurant_name_x", "restaurant_name_y"]
].drop_duplicates()


,restaurant_name_x,restaurant_name_y
1407,Hotel Dhaba Multicuisine,Restaurant_294


In [37]:
combo_revenue = df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum()
combo_revenue.sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [38]:
df["order_date"] = pd.to_datetime(df["order_date"])
df["quarter"] = df["order_date"].dt.to_period("Q")
quarter_revenue = df.groupby("quarter")["total_amount"].sum()
quarter_revenue
quarter_revenue.sort_values(ascending=False)


C:\Users\srini\AppData\Local\Temp\ipykernel_21880\3128716005.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [39]:
df[df["membership"] == "Gold"]["order_id"].count()


np.int64(4987)

In [40]:
round(df[df["city"] == "Hyderabad"]["total_amount"].sum())


1889367

In [41]:
df["user_id"].nunique()


2883

In [42]:
round(df[df["membership"] == "Gold"]["total_amount"].mean(), 2)


np.float64(797.15)

In [43]:
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")

df[df["rating"] >= 4.5]["order_id"].count()


np.int64(3374)

In [44]:
gold_df = df[df["membership"] == "Gold"]

gold_city_revenue = gold_df.groupby("city")["total_amount"].sum()
gold_city_revenue.sort_values(ascending=False)


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [45]:
gold_df[gold_df["city"] == "Chennai"]["order_id"].count()


np.int64(1337)

In [46]:
df.shape[0]


10000